In [ ]:
pip install rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import glob
import rasterio
import numpy as np

class Field:
    def __init__(self, name):
        self.name = name
        self.min_value = np.inf
        self.max_value = -np.inf
        self.available = True

def extract_field_range(file_path, field):
    with rasterio.open(file_path) as dataset:
        field_values = dataset.read(1)
        nodata_value = dataset.nodatavals[0]

        if nodata_value is not None:
            field_values = field_values[field_values != nodata_value]

        field_values = field_values.flatten()

        field.min_value = np.nanmin([field.min_value, np.nanmin(field_values)])
        field.max_value = np.nanmax([field.max_value, np.nanmax(field_values)])

        if field.min_value > field.max_value:
            field.available = False

directory = '/content/drive/MyDrive/Project_2/Insect Pest/tif files/US_Clip_From_World_Clim/'
tiff_files = glob.glob(directory + "/*.tif*")

fields = []
field_order = []

for tiff_file in tiff_files:
    file_name = os.path.basename(tiff_file)
    field_name = os.path.splitext(file_name)[0]
    field_order.append(field_name)  # Add field name to the order list
    field = Field(field_name)
    fields.append(field)

for tiff_file, field in zip(tiff_files, fields):
    extract_field_range(tiff_file, field)

for field in fields:
    print(f"{field.name}:")
    if field.available:
        print("\nMinimum value:", field.min_value)
        print("Maximum value:", field.max_value, "\n")
    else:
        print("\nRange not available")


